# step1: import important libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# step2:import the data

In [3]:
df=pd.read_csv(r"E:\my courses\courses\mine\ai\Machine Learning\machine learning\datasets\regularization\Advertising.csv")

# step3: data overview

In [4]:
df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


In [71]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   TV         200 non-null    float64
 1   radio      200 non-null    float64
 2   newspaper  200 non-null    float64
 3   sales      200 non-null    float64
dtypes: float64(4)
memory usage: 6.4 KB


# step4: EDA

# step5: Determine the features and target variable(label)

In [5]:
X=df.drop("sales",axis=1)
y=df["sales"]

# step6:Preprocessing(polynomial conversion)

In [7]:
from sklearn.preprocessing import PolynomialFeatures

In [8]:
polynomial_converter=PolynomialFeatures(degree=3,include_bias=False)

In [9]:
poly_features=polynomial_converter.fit_transform(X)

In [10]:
poly_features.shape

(200, 19)

# step7: split the data to train and test

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
X_train,X_test,y_train,y_test=train_test_split(poly_features,y,test_size=0.3,random_state=101)

# step8: scalling the data

In [13]:
from sklearn.preprocessing import StandardScaler

In [14]:
scaler=StandardScaler()

In [32]:
scaler.fit(X_train)

StandardScaler()

In [33]:
X_train=scaler.transform(X_train)
X_test=scaler.transform(X_test)

# Step9:regularization

## 1-ridge regression or L2

In [34]:
from sklearn.linear_model import Ridge

In [35]:
ridge_model=Ridge(alpha=10)
# alpha hyper paramtre==> pish farzesh adade 1 

In [36]:
ridge_model.fit(X_train,y_train)

Ridge(alpha=10)

In [37]:
y_pred=ridge_model.predict(X_test)

In [38]:
from sklearn.metrics import mean_absolute_error,mean_squared_error

In [39]:
MAE=mean_absolute_error(y_test,y_pred)
MSE=mean_squared_error(y_test,y_pred)
RMSE=np.sqrt(MSE)

In [40]:
pd.DataFrame([MAE,MSE,RMSE],index=["MAE","MSE","RMSE"],columns=["metrics"])

,metrics
MAE,0.577440
MSE,0.800378
RMSE,0.894639


# Ridge Regression with CV==> cross validation

In [41]:
from sklearn.linear_model import RidgeCV

In [42]:
ridge_cv_model=RidgeCV(alphas=(0.1,1.0,10.0),scoring="neg_mean_absolute_error")

In [43]:
ridge_cv_model.fit(X_train,y_train)

RidgeCV(scoring='neg_mean_absolute_error')

In [45]:
ridge_cv_model.alpha_

0.1

In [47]:
y_pred_ridge=ridge_cv_model.predict(X_test)

In [48]:
MAE_ridge=mean_absolute_error(y_test,y_pred_ridge)
MSE_ridge=mean_squared_error(y_test,y_pred_ridge)
RMSE_ridge=np.sqrt(MSE_ridge)

In [49]:
pd.DataFrame([MAE_ridge,MSE_ridge,RMSE_ridge],index=["MAE","MSE","RMSE"],columns=["Ridge metrics"])

,Ridge metrics
MAE,0.427377
MSE,0.382013
RMSE,0.618072


In [60]:
ridge_cv_model.coef_

array([ 5.40769393,  0.58858651,  0.40390394, -6.18263924,  4.59607939,
       -1.18789653, -1.15200458,  0.57837796, -0.12615859,  2.5569777 ,
       -1.38900471,  0.86059434,  0.72219553, -0.26129256,  0.17870786,
        0.44353612, -0.21362435, -0.04622473, -0.06441449])

# 2-Lasso regression or L1

In [50]:
from sklearn.linear_model import LassoCV

In [51]:
lasso_cv_model=LassoCV(eps=0.01,n_alphas=100,cv=5)
# 100 ta hyper paramtre ba ham moghayese konad
# fasele =eps

In [52]:
lasso_cv_model.fit(X_train,y_train)

LassoCV(cv=5, eps=0.01)

In [53]:
lasso_cv_model.alpha_

0.04943070909225771

In [54]:
y_pred_lasso=lasso_cv_model.predict(X_test)

In [57]:
MAE_lasso=mean_absolute_error(y_test,y_pred_lasso)
MSE_lasso=mean_squared_error(y_test,y_pred_lasso)
RMSE_lasso=np.sqrt(MSE_lasso)

In [58]:
pd.DataFrame([MAE_lasso,MSE_lasso,RMSE_lasso],index=["MAE","MSE","RMSE"],columns=["lasso metrics"])

,lasso metrics
MAE,0.515979
MSE,0.579949
RMSE,0.761544


In [59]:
lasso_cv_model.coef_

array([ 2.35600233,  0.21183181,  0.        , -0.        ,  3.78675114,
       -0.        ,  0.        ,  0.0624318 ,  0.        , -1.01152151,
       -0.        , -0.        ,  0.        ,  0.        , -0.        ,
        0.        ,  0.        ,  0.        ,  0.        ])

# 3-elastic net

In [61]:
from sklearn.linear_model import ElasticNetCV

In [62]:
elastic_model=ElasticNetCV(l1_ratio=[0.1,0.5,0.7,0.9,0.95,0.99,1],cv=5,max_iter=100000)

In [63]:
elastic_model.fit(X_train,y_train)

ElasticNetCV(cv=5, l1_ratio=[0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1],
             max_iter=100000)

In [65]:
elastic_model.l1_ratio_

1.0

In [66]:
y_pred_elastic=elastic_model.predict(X_test)

In [68]:
MAE_elastic=mean_absolute_error(y_test,y_pred_elastic)
MSE_elastic=mean_squared_error(y_test,y_pred_elastic)
RMSE_elastic=np.sqrt(MSE_elastic)

In [69]:
pd.DataFrame([MAE_elastic,MSE_elastic,RMSE_elastic],index=["MAE","MSE","RMSE"],columns=["elastic metrics"])

,elastic metrics
MAE,0.433503
MSE,0.367617
RMSE,0.606314


In [70]:
elastic_model.coef_

array([ 4.86023329,  0.12544598,  0.20746872, -4.99250396,  4.38026519,
       -0.22977201, -0.        ,  0.07267717, -0.        ,  1.77780246,
       -0.69614918, -0.        ,  0.12044132, -0.        , -0.        ,
       -0.        ,  0.        ,  0.        , -0.        ])

# finalizing the model

In [135]:
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import ElasticNetCV
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

df=pd.read_csv(r"E:\my courses\courses\mine\ai\Machine Learning\machine learning\datasets\regularization\Advertising.csv")

# Separate features and target variable
X = df.drop("sales", axis=1)
y = df["sales"]

# Create polynomial features
final_polynomial_converter = PolynomialFeatures(degree=3, include_bias=False)
final_polynomial_features = final_polynomial_converter.fit_transform(X)

# Scale the features
scaler = StandardScaler()
final_polynomial_features_scaled = scaler.fit_transform(final_polynomial_features)

# Initialize and fit the ElasticNetCV model
elastic_model = ElasticNetCV(l1_ratio=[0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1], cv=5, max_iter=100000)
elastic_model.fit(final_polynomial_features_scaled, y)

# Model is now trained on the entire dataset

ElasticNetCV(cv=5, l1_ratio=[0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1],
             max_iter=100000)

# save

In [152]:
from joblib import dump, load
from sklearn.preprocessing import StandardScaler

# Save the model and the polynomial converter
dump(elastic_model, r"E:\my courses\courses\projects\sale.joblib")
dump(final_polynomial_converter, r"E:\my courses\courses\projects\conv.joblib")
dump(scaler, r"E:\my courses\courses\projects\scaler.joblib")

['E:\\my courses\\courses\\projects\\scaler.joblib']

# load

In [154]:
# Load the saved model, polynomial converter, and scaler
load_converter = load(r"E:\my courses\courses\projects\conv.joblib")
load_polymodel = load(r"E:\my courses\courses\projects\sale.joblib")
load_scaler = load(r"E:\my courses\courses\projects\scaler.joblib")


# Predict for new data

In [155]:
# Prepare new data for prediction
campaign = [[144, 30, 12]]

# Transform the data using the polynomial converter
campaign_poly = load_converter.transform(campaign)

# Scale the polynomial features
campaign_poly_scaled = load_scaler.transform(campaign_poly)

# Make a prediction
prediction = load_polymodel.predict(campaign_poly_scaled)
print(prediction)

[15.79509232]


C:\Users\ASUS\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but PolynomialFeatures was fitted with feature names
  warnings.warn(
